In [1]:
import torch
import torch.optim as optim
import torch.utils.data as data
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
%matplotlib inline

from neural import MixqVAE


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Data Loaders

In [3]:
BATCH_SIZE = 128
SHUFFLE = True
NUM_WORKERS = 12

In [4]:
train_set_loader = data.DataLoader(
    datasets.MNIST('./data', train=True, transform=transforms.ToTensor(), download=True),
    batch_size=BATCH_SIZE,
    shuffle=SHUFFLE,
    num_workers=NUM_WORKERS,
)

test_set_loader = data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.ToTensor(), download=True),
    batch_size=BATCH_SIZE,
    shuffle=SHUFFLE,
    num_workers=NUM_WORKERS,
)

## Net and Optimizer

In [9]:
net = MixqVAE(1, 128, 32, 2, 8, rgb=False).to(device)
optimizer = optim.Adam(net.parameters(), lr=1e-4)

In [10]:
try:
    net.load_state_dict(torch.load(open('state_dict.pth', 'rb')))
    print('State Dict loaded from \'state_dict.pth\'')
except:
    pass

## Train Loop

In [11]:
def train(epochs=10):
    print('='*10, end='')
    print(' TRAIN', end=' ') 
    print('='*10, end='\n\n')
    net.train()

    for epoch in range(1, epochs+1):
        running_loss = 0

        for i, batch in enumerate(train_set_loader, 1):
            images, _ = batch
            images = images.to(device)
            
            # Zero grad
            optimizer.zero_grad()

            # Forward
            encoded, quantized, recon_x = net(images)
            # Compute Loss
            loss_value = net.loss_function(images, recon_x, encoded, quantized)
            running_loss += loss_value.item()
            # Backward
            loss_value.backward()
            # Update
            optimizer.step()

            if i % 100 == 0:
                print(f'==> EPOCH[{epoch}]({i}/{len(train_set_loader)}): LOSS: {loss_value.item()}')
            
        print(f'=====> EPOCH[{epoch}] Completed: Avg. LOSS: {running_loss/len(train_set_loader)}')
        print()


In [12]:
train(100)

========== TRAIN ==========

==> EPOCH[1](100/469): LOSS: 33984.44921875
==> EPOCH[1](200/469): LOSS: 24007.09375
==> EPOCH[1](300/469): LOSS: 19757.138671875


KeyboardInterrupt: 

In [ ]:
image, label = test_set_loader.dataset[10]

encoded, quantized, recon = net(image.unsqueeze(0))

recon = recon[0].squeeze()
plt.imshow(recon.detach().numpy(), cmap='Greys');

In [ ]:
net.vq.embeddings.weight[0, :]

In [ ]:
quantized.shape

In [ ]:
encodings = torch.zeros(28*28, 128)
encodings[14, 12] = 1
encodings[32, 1] = 1
quantized = encodings @ net.vq.embeddings.weight

In [ ]:
quantized = quantized.reshape((1, 28, 28, 8)).permute(0, 3, 1, 2)

In [ ]:
recon = net.decoder(quantized)[0].squeeze()
plt.imshow(recon.detach().numpy(), cmap='Greys');

In [ ]:
image, label = test_set_loader.dataset[20]
image = image.view(-1, 28*28).unsqueeze(0)

mu, sigma = net.encode(image)
z_2 = mu + sigma * torch.rand_like(sigma)

recon = net.decode(z_2)
print(label)
plt.imshow(recon.view(28, 28).detach().numpy(), cmap='Greys');

In [ ]:
z_3 = z_2 - z_1
recon = net.decode(z_3)
plt.imshow(recon.view(28, 28).detach().numpy(), cmap='Greys');

In [ ]:
torch.save(net.state_dict(), open('vqa_state_dict.pth', 'wb'))